## Constructing, loading and navigating compacted colored De Bruijn graphs with Pyfrost

Pyfrost is a Python library to create and navigate compacted colored De Bruijn graphs with a NetworkX-like API. Behind the scenes, Pyfrost powered by [Bifrost](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-02135-8), a memory efficient data structure and method to construct such graphs. This tutorial will show how to create or load existing graphs with Pyfrost, and how to navigate them.

### Download test data

First, let's download some assemblies as test data. We'll use two *C. portucalensis* genome assemblies as input for our graph.

In [1]:
from pathlib import Path
import ncbi_genome_download as ngd

Path("_data").mkdir(exist_ok=True)

ngd.download(groups="bacteria", assembly_accessions="GCF_001037585.2,GCF_014337275.1", human_readable=True, file_formats="fasta", output="_data")

0

### Creating and loading a graph

Pyfrost provides the functions, `build`, `build_from_refs`, `build_from_samples` and `load` to create or load a graph. All `build_*` functions accept a `k` and `g` parameter for the k-mer size and minimizer size respectively. The `load` function will obtain the stored k-mer and minimzer size.

**WARNING!** The k-mer and minimizer size are set globally. It's not supported to load/create different graphs with different k-mer sizes in the same process!

#### Creating a graph from genome assemblies

To create a colored De Bruijn graph from a collection of genome assemblies, use the `pyfrost.build_from_refs`.

In [2]:
import os
import pyfrost


cwd = Path.cwd()
ref_dir = Path("_data/human_readable/refseq/bacteria")
os.chdir(ref_dir)

# build_from_refs() only supports str objects, no Path objects
assembly_list = [str(p) for p in Path().glob("**/*.fna.gz")]
print(assembly_list)

g = pyfrost.build_from_refs(assembly_list)

os.chdir(cwd)

['Citrobacter/sp./BIDMC108/GCF_001037585.2_ASM103758v2_genomic.fna.gz', 'Citrobacter/freundii/MGH279/GCF_014337275.1_ASM1433727v1_genomic.fna.gz']


#### Creating a graph from Illumina reads

Pyfrost has a separate function `pyfrost.build_from_samples` to build a graph from read data sets. Since read data often contains sequencing errors, generating a lot of errorneous k-mers occuring only once, this function filters singleton k-mers. If you want to create a graph from both assemblies and reads, use `pyfrost.build`.

In [3]:
g = pyfrost.build_from_samples(['_data/Citr_sp_BIDMC108.subsampled.concat.fq.gz'])

#### Loading an existing graph created with `Bifrost`

If you want to load a graph created using the `Bifrost` CLI tool, use the `pyfrost.load` function, and point it to the GFA file generated by `Bifrost`. Pyfrost only supports **colored** De Bruijn graphs created by Bifrost (so enable the `-c` flag). To demonstrate how this works, let's build an example graph using `Bifrost` first.

In [4]:
%%bash
# Let's build a graph from both the assemblies and the read data using the Bifrost CLI tool
ls -1 _data/human_readable/refseq/bacteria/*/*/*/*.fna.gz > _data/ref_list.txt
Bifrost build -r _data/ref_list.txt -s _data/Citr_sp_BIDMC108.subsampled.concat.fq.gz -c -i -d -o _data/combined_graph

In the above example, we first built a text file listing the genome assemblies one per line, gave that file as input to the `Bifrost` command (`-r`), additionally specify the path to a read data set (`-s`), enable colors (`-c`), clip short tips (`-i`), and delete isolated unitigs (`-d`). We specify the output prefix as `_data/combined_graph`, resulting in the files `_data/combined_graph.gfa` and `_data/combined_graph.bfg_colors`. The latter file stores all graph color information.

To load the graph in Pyfrost, use the `load` function. You only need to specify the path to the GFA file, the path to bfg_colors file is automatically inferred.

In [5]:
g = pyfrost.load('_data/combined_graph.gfa')

#### A note on "colors"

One limitation of Bifrost (and therefore Pyfrost), is that there are a limited number of ways to specify which files or sequences should belong to which colors. In Bifrost, each file is one color. Thus, if you have a paired Illumina data, usually split in two files, we need to concatenate the two FASTQ files to ensure that k-mers from that sample obtain the same colors. On the other hand, in case of genome assemblies, each contig is listed in a single FASTA file, and all contigs will thus get the same color. If you want each contig to have a separate color you'll have to split them in individual files.

### Using the graph

The resulting graph object mimics a NetworkX's [DiGraph](https://networkx.org/documentation/stable/reference/classes/digraph.html). Thus many standard algorithms included with NetworkX will function automatically.

Due to the double stranded nature of DNA, the graph is *[skew symmetric](https://en.wikipedia.org/wiki/Skew-symmetric_graph)*, i.e., for every node (unitig) $v$, it's reverse complement $\bar{v}$ is also a node. And for every incoming edge $(u, v)$, an equivalent outgoing edge $(\bar{v}, \bar{u})$ exists (and vice versa). Under the hood it's all stored efficiently in memory, but you'll have to take this into account when listing nodes and edges.

#### Graph attributes

To obtain the k-mer size and color names, use the `graph` attribute.

In [6]:
g.graph

{'color_names': ['_data/Citr_sp_BIDMC108.subsampled.concat.fq.gz',
  '_data/human_readable/refseq/bacteria/Citrobacter/freundii/MGH279/GCF_014337275.1_ASM1433727v1_genomic.fna.gz',
  '_data/human_readable/refseq/bacteria/Citrobacter/sp./BIDMC108/GCF_001037585.2_ASM103758v2_genomic.fna.gz'],
 'k': 31,
 'g': 23}

### Access unitigs and k-mers

#### Unitigs

Each node represents a unitig and is keyed by the first k-mer of that unitig. Thus, if you list the first 10 nodes, you'll get back a list of 10 k-mers representing the unitig's heads.

In [7]:
list(g.nodes)[:10]

[<Kmer 'GCGACCGTTACTCGCTGGGGGTAAGAATTAT'>,
 <Kmer 'CCGACTTCAAAGCGCAGCTGTGGGGCATCCG'>,
 <Kmer 'GGATGCCCCACAGCTGCGCTTTGAAGTCGGT'>,
 <Kmer 'CGTTTTCAGCGTTTGCGTGACGGGTTTAGTA'>,
 <Kmer 'ACTAAACCCGTCACGCAAACGCTGAAAACGC'>,
 <Kmer 'GGTTGCACTTGCTGAGCCACGTGGGCGGGCG'>,
 <Kmer 'GCCCGCCCACGTGGCTCAGCAAGTGCAACCA'>,
 <Kmer 'GCCTGGACGCGCCGCCGGAGCAGCACGTTGT'>,
 <Kmer 'CAACGTGCTGCTCCGGCGGCGCGTCCAGGCT'>,
 <Kmer 'CCATATAGGAATAACGGATTGTAGGCGCCGC'>]

In [8]:
len(g.nodes)

134100

`g.nodes` acts as a dict-of-dicts for node attributes and accepts both `Kmer` objects or strings of length `k`. For example, the full unitig sequence of a node can be obtained using the 'unitig_sequence' attribute:

In [9]:
g.nodes['GCGACCGTTACTCGCTGGGGGTAAGAATTAT']['unitig_sequence']

'GCGACCGTTACTCGCTGGGGGTAAGAATTATACGGGCTGGTGGATAAAGCGCAAGGATCGCACGGGATCTTTATTAGATCGTTTAAGCAGTAAAGCATCTTTACTCATTAATTTTTTCAATATGCGCGCTAAAACCTGCAGCACTTCTCCAGGATCGTTTACACTTAGCCGATTCTGGATCATCCTGTGGATAAATCGGGAAGAATCTGTGAGAAACAGAAGATCTCTTTCTCAGTTTACGCTATGATCCGCGGTCCTGATCGTTTCAATGGATCCTGATCGGGTAAAAATTGCAGATAGCAATTCGCACGTCACCCTTTGCATAGGGTCTTGTCGATGTGCGTCAACAATCATGAATGTTTCAGCCTTTGTCATTTATCGACTTTTGTTCGAGTGGAGTCCGCCGTGTCACTTTCGCTTTGGCAGCAGTGTCTTGCCCGATTGCAGGATGAGTTACCAGCCACAGAGTTCAGTATGTGGATACGCCCGTTGCAGGCGGAACTGAGCGATAACACGCTGGCTTTGTATGCGCCAAACCGTTTCGTGCTCGATTGGGTAAGGGATAAATACCTTAATAATATTAATGGATTACTCAATAATTTCTGTGGAGCGGATGCCCCACAGCTGCGCTTTGAAGTCGG'

In [10]:
g.nodes['GCGACCGTTACTCGCTGGGGGTAAGAATTAT']['length']  # length in number of k-mers!

611

In [11]:
# head and tail k-mers of this unitig
g.nodes['GCGACCGTTACTCGCTGGGGGTAAGAATTAT']['head'], g.nodes['GCGACCGTTACTCGCTGGGGGTAAGAATTAT']['tail']

(<Kmer 'GCGACCGTTACTCGCTGGGGGTAAGAATTAT'>,
 <Kmer 'CGGATGCCCCACAGCTGCGCTTTGAAGTCGG'>)

To obtain the reverse complement node $\bar{v}$ for any node $v$, use the `twin` method, which returns the head k-mer of the reverse complement unitig.

In [12]:
twin = g.nodes['GCGACCGTTACTCGCTGGGGGTAAGAATTAT'].twin()

print(g.nodes[twin]['head'], g.nodes[twin]['tail'])

CCGACTTCAAAGCGCAGCTGTGGGGCATCCG ATAATTCTTACCCCCAGCGAGTAACGGTCGC


#### Finding specific k-mers

To find any specific k-mer (which may or may not be the head k-mer of a unitig), use `g.find`.

In [13]:
mapping = g.find('AATTATACGGGCTGGTGGATAAAGCGCAAGG')

The resulting mapping object has the same attributes as the unitig attributes obtained using `g.nodes`. There are a few additions, e.g., to obtain the position of the mapped k-mer within the unitig.

In [14]:
print("unitig k-mer:", mapping['head'], "position:", mapping['pos'], "length (number of k-mers):", mapping['length'])

subseq = g.nodes[mapping['head']]['unitig_sequence'][mapping['pos']:mapping['pos']+31]
print(subseq == mapping['mapped_sequence'])

unitig k-mer: GCGACCGTTACTCGCTGGGGGTAAGAATTAT position: 25 length (number of k-mers): 1
True


### Unitig and k-mer colors

Each k-mer can have associated "colors", which represent the original file sources when the graph was created. To see which samples or genome assemblies contain k-mers of a particular unitig, use the 'colors' attribute. This works for both node attributes as well as k-mer mappings. In case of a node attribute (thus representing a whole unitig), the color set represents the union of all colors of all k-mers of that particular unitig.

In [15]:
from pyfrost import Kmer

node = Kmer("GGTTGCACTTGCTGAGCCACGTGGGCGGGCG")

print(set(g.nodes[node]['colors']))

for c, color_name in enumerate(g.graph['color_names']):
    if c in g.nodes[node]['colors']:
        print("Present in", color_name)
    else:
        print("Not present in", color_name)

{1, 2}
Not present in _data/Citr_sp_BIDMC108.subsampled.concat.fq.gz
Present in _data/human_readable/refseq/bacteria/Citrobacter/freundii/MGH279/GCF_014337275.1_ASM1433727v1_genomic.fna.gz
Present in _data/human_readable/refseq/bacteria/Citrobacter/sp./BIDMC108/GCF_001037585.2_ASM103758v2_genomic.fna.gz


To obtain the associated colors of the k-mer at a specific position within the unitig, you can use indexing. Each time you use indexing, however, a new proxy object is constructed so it might not be the best for performance-critical applications.

In [16]:
print(set(g.nodes[node]['colors'][0]))
print(set(g.find(node)['colors']))  # equivalent because `node` is the first k-mer of this unitig

{1, 2}
{1, 2}


### Neighbors and navigation

Just as in regular NetworkX DiGraphs, you can use `g.succ` (or `g.adj`) and `g.pred` to access edges to a node's neighbors. `g.successors` (or `g.neighbors`) and `g.predecessors` all work as you'd expect. In addition to these standard NetworkX functions, pyfrost also provides `g.color_restricted_successors` and `g.color_restricted_predecessors`, which will only yield neighbors with a specific color.

In [17]:
i = 0
for n, neighbors in g.succ.items():
    if i == 5:
        break
    
    print("Node", n, "...", g.nodes[n]['tail'])
    for v in neighbors:
        print(v[-1], ":", g.nodes[v]['head'])
        
    print()
    i += 1

Node GCGACCGTTACTCGCTGGGGGTAAGAATTAT ... CGGATGCCCCACAGCTGCGCTTTGAAGTCGG
C : GGATGCCCCACAGCTGCGCTTTGAAGTCGGC
T : GGATGCCCCACAGCTGCGCTTTGAAGTCGGT

Node CCGACTTCAAAGCGCAGCTGTGGGGCATCCG ... ATAATTCTTACCCCCAGCGAGTAACGGTCGC
C : TAATTCTTACCCCCAGCGAGTAACGGTCGCC
T : TAATTCTTACCCCCAGCGAGTAACGGTCGCT

Node GGATGCCCCACAGCTGCGCTTTGAAGTCGGT ... TACTAAACCCGTCACGCAAACGCTGAAAACG
C : ACTAAACCCGTCACGCAAACGCTGAAAACGC

Node CGTTTTCAGCGTTTGCGTGACGGGTTTAGTA ... ACCGACTTCAAAGCGCAGCTGTGGGGCATCC
G : CCGACTTCAAAGCGCAGCTGTGGGGCATCCG

Node ACTAAACCCGTCACGCAAACGCTGAAAACGC ... CGCCCGCCCACGTGGCTCAGCAAGTGCAACC
A : GCCCGCCCACGTGGCTCAGCAAGTGCAACCA
G : GCCCGCCCACGTGGCTCAGCAAGTGCAACCG



### Other useful functions

#### Paths

* `pyfrost.path_sequence(g, path)` - Build to full DNA sequence represented by the given path
* `pyfrost.path_nucleotide_length(g, path)` - Calculate the full path in number of nucleotides for the given path
* `pyfrost.path_rev_compl(g, path)` - Get the reverse complement path
* `pyfrost.path_kmers(g, path)` - Get all k-mers for a given path (also internal unitig k-mers)

#### Misc sequence utilities

* `pyfrost.kmerize_str(seq)` - K-merize a string
* `pyfrost.reverse_complement(seq)` - Get the reverse complement of a sequence